In [ ]:
# | default_exp _code_generator.test_generator

In [ ]:
# | export

from typing import *
import time
import importlib.util
from tempfile import TemporaryDirectory
from pathlib import Path
import platform
import subprocess  # nosec: B404: Consider possible security implications associated with the subprocess module.

from yaspin import yaspin

from faststream_gen._components.logger import get_logger
from faststream_gen._code_generator.helper import (
    CustomAIChat,
    ValidateAndFixResponse,
    write_file_contents,
    read_file_contents,
    validate_python_code,
)
from faststream_gen._code_generator.prompts import TEST_GENERATION_PROMPT
from faststream_gen._code_generator.constants import (
    APPLICATION_FILE_NAME,
    INTEGRATION_TEST_FILE_NAME,
)

In [ ]:


from faststream_gen._components.logger import suppress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export


def _validate_response(test_code: str, **kwargs: str) -> List[str]:
    with TemporaryDirectory() as d:
        write_file_contents(f"{d}/{APPLICATION_FILE_NAME}", kwargs["app_code"])
        
        test_file = f"{d}/{INTEGRATION_TEST_FILE_NAME}"
        write_file_contents(test_file, test_code)

#         cmd = ["python3", test_file]
        cmd = ["pytest", test_file, "--tb=short"]
        # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
        p = subprocess.run(  # nosec: B602, B603 subprocess call - check for execution of untrusted input.
            cmd,
            stderr=subprocess.PIPE,
            stdout=subprocess.PIPE,
            shell=True if platform.system() == "Windows" else False,
        )
        if p.returncode != 0:
            return [str(p.stdout.decode('utf-8'))]

        return []

In [ ]:
fixture_test_code = """
def test_always_passes():
    assert True
"""
kwargs = {
    "app_code": "print('hi')"
}
expected = []
actual = _validate_response(fixture_test_code, **kwargs)
print(actual)
assert actual == expected

[]


In [ ]:
fixture_test_code = """
def test_always_fails():
    assert False
"""
kwargs = {
    "app_code": "print('hi')"
}
actual = _validate_response(fixture_test_code, **kwargs)
print(actual[0])
assert actual != []
print("OK")

============================= test session starts ==============================
platform linux -- Python 3.11.4, pytest-7.4.0, pluggy-1.3.0
rootdir: /tmp/tmpv2jlqi2_
plugins: anyio-3.7.1, asyncio-0.21.1
asyncio: mode=Mode.STRICT
collected 1 item

../../../tmp/tmpv2jlqi2_/test.py F                                       [100%]

=================================== FAILURES ===================================
______________________________ test_always_fails _______________________________
/tmp/tmpv2jlqi2_/test.py:3: in test_always_fails
    assert False
E   assert False
=========================== short test summary info ============================
FAILED ../../../tmp/tmpv2jlqi2_/test.py::test_always_fails - assert False
============================== 1 failed in 0.04s ===============================

OK


In [ ]:
# fixture_test_code = """
# import asyncio
# from fastkafka.testing import Tester
# try:
#     from .application import *
# except ImportError as e:
#     from application import *


# async def async_tests():
#     async with Tester(change_currency_app) as tester:
#         input_msg = StoreProduct(product_name="Test Product", currency="HRK", price=100.0)

#         # tester consumes message from the store_product topic
#         await tester.on_store_product(input_msg)

#         # assert that tester consumed from the store_product topic and it was called with the accurate argument
#         await tester.awaited_mocks.on_store_product.assert_called_with(
#             input_msg, timeout=5
#         )

#         # assert that tester produced message to the change_currency topic and it was called with the accurate argument
#         await tester.awaited_mocks.to_change_currency.assert_called_with(
#             StoreProduct(product_name="Test Product", currency="EUR", price=13.333333333333334), timeout=5
#         )
#     print("ok")


# if __name__ == "__main__":
#     loop = asyncio.get_event_loop()
#     loop.run_until_complete(async_tests())

# """

# kwargs = {
#     "app_code": """
# from typing import *
# from pydantic import BaseModel, Field
# from aiokafka.helpers import create_ssl_context

# from fastkafka import FastKafka


# class StoreProduct(BaseModel):
#     product_name: str = Field(..., description="Name of the product.")
#     currency: str = Field(..., description="Currency of the product.", pattern="^[A-Z]{3}$")
#     price: float = Field(..., description="Price of the product.")

# kafka_brokers = {
#     "localhost": {
#         "url": "localhost",
#         "description": "local development kafka broker",
#         "port": 9092,
#     },
#     "staging": {
#         "url": "staging.airt.ai",
#         "description": "staging kafka broker",
#         "port": 9092,
#         "protocol": "kafka-secure",
#         "security": {"type": "scramSha256"},
#     },
#     "production": {
#         "url": "prod.airt.ai",
#         "description": "production kafka broker",
#         "port": 9092,
#         "protocol": "kafka-secure",
#         "security": {"type": "scramSha256"},
#     }
# }

# change_currency_app_description = "Create a FastKafka application using localhost broker for testing, staging.airt.ai for staging and prod.airt.ai for production. Use default port number. It should consume messages from 'store_product' topic and the message will be a JSON encoded object with three attributes: product_name, currency, and price. For each consumed message, check if the currency attribute is set to 'HRK'. If it is, change the currency to 'EUR' and divide the price by 7.5. If the currency is not set to 'HRK', the original message remains unchanged. Finally, publish the consumed message to 'change_currency' topic."

# change_currency_app = FastKafka(
#     kafka_brokers=kafka_brokers, 
#     description=change_currency_app_description, 
#     version="0.0.1", 
#     title='Change Currency',
#     security_protocol = "SASL_SSL",
#     sasl_mechanism= "SCRAM-SHA-256",
#     sasl_plain_username= "<username>",
#     sasl_plain_password=  "<password>",
#     ssl_context= create_ssl_context(),
# )


# store_product_description = "For each consumed message, check if the currency attribute is set to 'HRK'. If it is, change the currency to 'EUR' and divide the price by 7.5. If the currency is not set to 'HRK', the original message remains unchanged. Finally, publish the consumed message to 'change_currency' topic."

# @change_currency_app.consumes(topic="store_product", description=store_product_description)
# async def on_store_product(msg: StoreProduct):
#     if msg.currency == "HRK":
#         msg.currency = "EUR"
#         msg.price /= 7.5
#     await to_change_currency(msg)


# change_currency_description = "Produce the incoming messages to the 'change_currency' topic."
# @change_currency_app.produces(topic="change_currency", description=change_currency_description)
# async def to_change_currency(msg: StoreProduct) -> StoreProduct:
#     return msg

# """
# }

# expected = 'AttributeError: \'Tester\' object has no attribute \'on_store_product\''
# actual = _validate_response(fixture_test_code, **kwargs)
# print(actual)
# assert expected in "".join(actual)

In [ ]:
# | export


def generate_test(
    description: str, code_gen_directory: str, total_usage: List[Dict[str, int]]
) -> List[Dict[str, int]]:
    """Generate integration test for the FastKafka app

    Args:
        description: Validated User application description
        code_gen_directory: The directory containing the generated files.

    Returns:
        The generated integration test code for the application
    """
    with yaspin(text="Generating tests (usually takes around 10 to 30 seconds)...", color="cyan", spinner="clock") as sp:
        app_file_name = f"{code_gen_directory}/{APPLICATION_FILE_NAME}"
        app_code_prompt = read_file_contents(app_file_name)

        prompt = TEST_GENERATION_PROMPT.replace(
            "==== REPLACE WITH APP DESCRIPTION ====", description
        )
        test_generator = CustomAIChat(user_prompt=prompt)
        test_validator = ValidateAndFixResponse(test_generator, _validate_response)
        validated_test, total_usage = test_validator.fix(
             f"{TEST_GENERATION_PROMPT}\n{app_code_prompt}", total_usage=total_usage, app_code=app_code_prompt
        )

        output_file = f"{code_gen_directory}/{INTEGRATION_TEST_FILE_NAME}"
        write_file_contents(output_file, validated_test)

        sp.text = ""
        sp.ok(f" ✔ Tests are generated and saved at: {output_file}")
        return total_usage

In [ ]:
# | notest

fixture_code = """
from pydantic import BaseModel, Field

from faststream import FastStream, Logger
from faststream.kafka import KafkaBroker


class Product(BaseModel):
    product_name: str = Field(
        ..., examples=["Apple"], description="Product name example"
    )
    currency: str = Field(
        ..., examples=["HRK"], description="Currency example"
    )
    price: float = Field(
        ..., examples=[10.0], description="Price example"
    )


broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


@broker.publisher("change_currency")
@broker.subscriber("store_product")
async def on_store_product(msg: Product, logger: Logger) -> Product:
    logger.info(msg)

    if msg.currency == "HRK":
        logger.info(f"Changing currency and price for {msg.product_name}")
        msg = Product(product_name=msg.product_name, currency="EUR", price = msg.price / 7.5)

    return msg
"""

fixture_description = """
Create a FastStream application using localhost broker for testing and use default port number. It should consume from 'store_product' topic an JSON encoded object with the following three attributes: product_name, currency and price. The format of the currency will be three letter string, e.g. 'EUR'. For each consumed message, check if the currency attribute is set to 'HRK'. If it is then change the currency to 'EUR' and divide the price by 7.5, if the currency is not set to 'HRK' don't change the original message. Finally, publish the consumed message to 'change_currency' topic.
"""

with TemporaryDirectory() as d:
    output_path = f"{str(d)}/fastkafka-gen"
    test_file = f"{output_path}/{APPLICATION_FILE_NAME}"    
    write_file_contents(test_file, fixture_code)
    
    usage = generate_test(fixture_description, output_path, [])
    
    assert Path(output_path).exists()
    
    actual = [file for file in Path(output_path).iterdir()]
    print(actual)
    assert len(actual) == 2
    
    output_file = f"{output_path}/{INTEGRATION_TEST_FILE_NAME}"
    contents = read_file_contents(output_file)
    print(contents)

assert int(usage[0]["total_tokens"]) > 0
print(usage)

⠹ Generating tests (usually takes around 10 to 30 seconds)... 

/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:120: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(color) if color else color


 ✔ Tests are generated and saved at: /tmp/tmpg_fxihc1/fastkafka-gen/test.py 
[PosixPath('/tmp/tmpg_fxihc1/fastkafka-gen/application.py'), PosixPath('/tmp/tmpg_fxihc1/fastkafka-gen/test.py')]
import pytest

from faststream.kafka import TestKafkaBroker

from application import *


@broker.subscriber("change_currency")
async def on_change_currency(msg: Product):
    pass

@pytest.mark.asyncio
async def test_app():
    async with TestKafkaBroker(broker):
        await broker.publish(Product(product_name="Apple", currency="HRK", price=10.0), "store_product")
        on_store_product.mock.assert_called_with(dict(Product(product_name="Apple", currency="HRK", price=10.0)))
        on_change_currency.mock.assert_called_with(dict(Product(product_name="Apple", currency="EUR", price=1.3333333333333333)))
[defaultdict(<class 'int'>, {'prompt_tokens': 6362, 'completion_tokens': 138, 'total_tokens': 6500})]
